<a href="https://colab.research.google.com/github/HiranmaiKaredla/NLP/blob/main/doc_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -U scikit-learn
!pip install wikipedia

from collections import OrderedDict
import multiprocessing
import psutil
import pandas as pd
pd.set_option('display.max_rows', None)
import sklearn
import re
import numpy as np
import wikipedia
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
print('Import successful')




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=c1e3fce6e34346e4e2694278a58639daa22e4f72d3c98723086ca7ec72e147bb
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia
Import successful


In [ ]:
# Build dataset

# Get pages names
search_term = "science"
pages = wikipedia.search("science", results=100)
print('pages', len(pages), pages[:20])

pages 100 ['Science', 'Science (disambiguation)', 'Science fiction', 'Computer science', 'Bachelor of Science', 'Political science', 'Social science', 'Natural science', 'Science (journal)', 'Data science', 'Science in science fiction', 'Pseudoscience', 'Botany', 'Logic', 'Applied science', 'Master of Science', 'Forensic science', 'Medicine', 'Science of team science', 'E-Science']


In [ ]:
# Get page content
documents = OrderedDict()
for i, p in tqdm(enumerate(pages), desc='fetching docs'):
    try:
        text = wikipedia.summary(p, sentences=1)
        documents[p] = text
    except Exception as e:
        # print('Could not load:', p)
        pass


print('pages', len(documents), list(documents.keys())[:20])
print()

for p, text in list(documents.items())[:10]:
    print()
    print(f'{p}: {text[:100]}...')




fetching docs: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.9/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


pages 88 ['Science fiction', 'Computer science', 'Bachelor of Science', 'Political science', 'Social science', 'Natural science', 'Science (journal)', 'Data science', 'Science in science fiction', 'Pseudoscience', 'Botany', 'Applied science', 'Master of Science', 'Forensic science', 'Medicine', 'Science of team science', 'E-Science', 'Chemistry', 'Military science', 'Actuarial science']


Science fiction: Science fiction (sometimes shortened to sf or sci-fi) is a genre of speculative fiction, which typic...

Computer science: Computer science is the study of computation, automation, and information....

Bachelor of Science: A Bachelor of Science (BS, BSc, SB, or ScB; from the Latin scientiae baccalaureus) is a bachelor's d...

Political science: Political science is the scientific study of politics....

Social science: Social science is one of the branches of science, devoted to the study of societies and the relation...

Natural science: Natural science is one of the branches of scien

In [ ]:
# Preprocessing
text_clean = []
for k, text in documents.items():
    text_clean.append(re.sub('[^a-zA-Z]', ' ', text))

for t in text_clean[:5]:
    print()
    print(t)


Science fiction  sometimes shortened to sf or sci fi  is a genre of speculative fiction  which typically deals with imaginative and futuristic concepts such as advanced science and technology  space exploration  time travel  parallel universes  and extraterrestrial life 

Computer science is the study of computation  automation  and information 

A Bachelor of Science  BS  BSc  SB  or ScB  from the Latin scientiae baccalaureus  is a bachelor s degree awarded for programs that generally last three to five years The first university to admit a student to the degree of Bachelor of Science was the University of London in      

Political science is the scientific study of politics 

Social science is one of the branches of science  devoted to the study of societies and the relationships among individuals within those societies 


In [ ]:
# create tfidf vectorizer
vectorizer = TfidfVectorizer( \
    stop_words = 'english',
    lowercase = True,
    #min_df refers words occur in 3 distinct documents,
    min_df = 3 )


In [ ]:
# 'train' vectorizer on corpus
vectorizer.fit(text_clean)

feature_names = vectorizer.get_feature_names_out()
print(feature_names)
print(len(feature_names))


['academic' 'academy' 'advanced' 'ancient' 'application' 'awarded'
 'bachelor' 'best' 'branch' 'branches' 'college' 'computer' 'computing'
 'concerned' 'data' 'deals' 'degree' 'discipline' 'disciplines' 'earth'
 'education' 'empirical' 'engineering' 'english' 'extraterrestrial' 'fi'
 'fiction' 'field' 'fields' 'film' 'films' 'futuristic' 'genre' 'greek'
 'group' 'history' 'human' 'imaginative' 'includes' 'including'
 'information' 'interdisciplinary' 'knowledge' 'known' 'life' 'list'
 'london' 'management' 'mathematics' 'medicine' 'method' 'methods'
 'natural' 'organization' 'phenomena' 'philosophy' 'physical' 'political'
 'practices' 'primarily' 'problems' 'processes' 'public' 'refer'
 'referred' 'related' 'research' 'school' 'sci' 'science' 'sciences'
 'scientific' 'shortened' 'social' 'speculative' 'studies' 'study'
 'systems' 'technical' 'technology' 'term' 'time' 'travel' 'typically'
 'universities' 'university' 'use' 'used' 'uses' 'various' 'world' 'years']
92


In [ ]:
# get tfidf values
X = vectorizer.transform(text_clean)
print('X content', X)
print('X.shape', X.shape)

# (0, 83)	0.21652613747132152
# (index of doc, word number)

X content   (0, 83)	0.21652613747132152
  (0, 82)	0.2507290189478579
  (0, 81)	0.22594759688209073
  (0, 79)	0.18890148150231345
  (0, 74)	0.23709081823516814
  (0, 72)	0.2507290189478579
  (0, 69)	0.14985726106688477
  (0, 68)	0.23709081823516814
  (0, 44)	0.21652613747132152
  (0, 37)	0.2507290189478579
  (0, 32)	0.21652613747132152
  (0, 31)	0.2507290189478579
  (0, 26)	0.36716695208983136
  (0, 25)	0.2507290189478579
  (0, 24)	0.2507290189478579
  (0, 15)	0.2507290189478579
  (0, 2)	0.2507290189478579
  (1, 76)	0.40769025851620105
  (1, 69)	0.2084633748948872
  (1, 40)	0.6286221736617421
  (1, 11)	0.6286221736617421
  (2, 91)	0.22639393132110128
  (2, 85)	0.42815883574736047
  (2, 69)	0.13531251632664082
  (2, 46)	0.22639393132110128
  :	:
  (82, 10)	0.24038225258661863
  (83, 79)	0.5635433451854571
  (83, 71)	0.463890524416264
  (83, 69)	0.22353202719833712
  (83, 65)	0.6459550388924837
  (84, 49)	0.7883306277633328
  (84, 47)	0.34039575194811117
  (84, 15)	0.3941653138816664
  (8

In [ ]:
# Highestest weighted features
i_max = X.argmax(axis=1)
print('i_max shape', i_max.shape)

page_names = list(documents.keys())

# Iterate over documents
rows = []
for j, i in enumerate(i_max):
    rows.append(dict(page=page_names[j], top_feature=feature_names[i].item()))
df = pd.DataFrame(rows)
print(df)

i_max shape (88, 1)
                                                 page        top_feature
0                                     Science fiction            fiction
1                                    Computer science           computer
2                                 Bachelor of Science           bachelor
3                                   Political science          political
4                                      Social science             social
5                                     Natural science            natural
6                                   Science (journal)           academic
7                                        Data science               data
8                          Science in science fiction            fiction
9                                       Pseudoscience         scientific
10                                             Botany               life
11                                    Applied science             method
12                             

In [ ]:
# Get page names and text separately
page_names = list(documents.keys())
text_original = list(documents.values())

similarity = {}
N = len(text_clean)
for i in range(N):
    for j in range(i+1,N):
        s = cosine_similarity(X[i], X[j])
        similarity[(i, j)] = s

        if (i < 5) and (j < 5):
            print()
            print(f'similarity = {s[0]}')
            print(f'page={i}, title={page_names[i]}, text={text_original[i]}')
            print(f'page={j}, title={page_names[j]}, text={text_original[j]}')


similarity = [0.03123975]
page=0, title=Science fiction, text=Science fiction (sometimes shortened to sf or sci-fi) is a genre of speculative fiction, which typically deals with imaginative and futuristic concepts such as advanced science and technology, space exploration, time travel, parallel universes, and extraterrestrial life.
page=1, title=Computer science, text=Computer science is the study of computation, automation, and information.

similarity = [0.02027756]
page=0, title=Science fiction, text=Science fiction (sometimes shortened to sf or sci-fi) is a genre of speculative fiction, which typically deals with imaginative and futuristic concepts such as advanced science and technology, space exploration, time travel, parallel universes, and extraterrestrial life.
page=2, title=Bachelor of Science, text=A Bachelor of Science (BS, BSc, SB, or ScB; from the Latin scientiae baccalaureus) is a bachelor's degree awarded for programs that generally last three to five years.The first u

In [ ]:
'''

Determine the top 5 wikipedia pages that are most similar to
'Information science' based on the summary text.  Ignore
pages with summaries that are identical to 'Science and technology'.
What is the hi features for 'Science and technology' and the top 5
matches have the highest TF-IDF values?
'''
target = 'Information science'

In [ ]:
print(target in page_names)

True
